In [1]:
from nltk.tokenize import RegexpTokenizer
import cPickle as pickle
import string
import os
import re
import urllib2

import numpy as np
import random
import time

import json

import logging
from logging import info

from multiprocessing import Pool as ThreadPool
import itertools

import xml.etree.ElementTree as ET

import nltk

from thesis.utils.text import get_sentences, sentence_wordtokenizer

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
# root_location = "/mnt/data2/shalaby/"
root_location = "/home/local/shalaby/"
exports_location = root_location + "exported_data/"

# training_file = root_location + 'docs_output_training_validation_documents_' + str(SAMPLE_RATIO)
training_file = root_location + 'docs_output.json'

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
sections_file = exports_location + "sections.pkl"
classes_file = exports_location + "classes.pkl"
subclasses_file = exports_location + "subclasses.pkl"
classifications_output = exports_location + "classifications.pkl"
# training_docs_list_file = exports_location + "training_documents_" + str(SAMPLE_RATIO) + "_sample.pkl"
# validation_docs_list_file = exports_location + "validation_documents_" + str(SAMPLE_RATIO) + "_sample.pkl"
training_docs_list_file = exports_location + "extended_pv_training_docs_list.pkl"
validation_docs_list_file = exports_location + "extended_pv_validation_docs_list.pkl"
test_docs_list_file = exports_location + "extended_pv_test_docs_list.pkl"

In [4]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
classes = pickle.load(open(classes_file))
subclasses = pickle.load(open(subclasses_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 25 s, sys: 3.84 s, total: 28.8 s
Wall time: 29.2 s


In [5]:
len(training_docs_list)

120156

#### Extraction Utils 

In [6]:
#ES_URL = 'http://localhost:9200/patents/patent/{}'
ES_URL = 'http://yell.dbs.ifi.lmu.de:9200/patents/patent/{}'
HEADING_TAG = 'heading'
PARAGRAPH_TAG = 'p'
UL_TAG = 'ul'
LI_TAG = 'li'
OL_TAG = 'ol'
DESC_OF_DRAWINGS_TAG = 'description-of-drawings'
MIN_PARAGRAPH_LENGTH = 50

In [7]:
def merge_with_previous(curr_node_tag, previous_node_tag, previous_node_text):
    if curr_node_tag == PARAGRAPH_TAG and previous_node_tag == HEADING_TAG:
        return True
    if previous_node_text and len(previous_node_text) < MIN_PARAGRAPH_LENGTH:
        return True
    return False
    
def get_paragraphs(root):
    paragraphs = []
    previous_node_text = None
    previous_tag = None
    for child in root:
        node_text = None
        if child.tag != DESC_OF_DRAWINGS_TAG:
            node_text = get_node_text(child)
            if node_text.strip():
                if merge_with_previous(child.tag, previous_tag, previous_node_text) and len(paragraphs) > 0:
                    paragraphs[-1] += ' ' + node_text
                else:
                    paragraphs.append(node_text)
        else:
            node_text = extract_desc_of_drawings_paragraph(child)
            paragraphs.append(node_text)
            
        previous_tag = child.tag
        previous_node_text = node_text
    return paragraphs
    
def extract_desc_of_drawings_paragraph(node):
    previous_tag = None
    sentences = []
    for child in node:
        node_text = get_node_text(child)
        if child.tag == PARAGRAPH_TAG and previous_tag == HEADING_TAG:
            sentences[-1] += ' ' + node_text
        else:
            # a paragraph in drawings descriptions is treated as a sentence
            if child.tag == PARAGRAPH_TAG:
                node_text = apply_sentence_end(node_text)
            sentences.append(node_text)
        previous_tag = child.tag
    
    return ' '.join(sentences)

def apply_sentence_end(text):
    if text and text.strip():
        text = text.strip().strip(';.')
        text += '. '
    return text

def itertext_custom(self):
    tag = self.tag
    if not isinstance(tag, basestring) and tag is not None:
        return
    if self.text:
        if tag == LI_TAG:
            yield apply_sentence_end(self.text)
        else:
            yield self.text.replace('\n',' ')
    for e in self:
        for s in e.itertext_custom():
            yield s
        if e.tail:
            yield e.tail

ET.Element.itertext_custom = itertext_custom
# def get_node_text(node):
#     node_text = ''
#     for child in node:
#         # for ul tags, get li tags as sentences
#         if child.tag == UL_TAG:
#             li_sentences = [apply_sentence_end(get_node_text_iterative(c)) for c in child]
#             child_text = ' '.join(li_sentences)
#         else:
#             child_text = get_node_text_iterative(child)
#         node_text += child_text
#     return node_text
        
get_node_text = lambda node: ''.join(node.itertext_custom()).strip()

In [54]:
def conc_paragraphs(parag1, parag2):
    return parag1.strip('.') + '.' + ' ' + parag2

def concatenate_sentences_to_paragraphs(paragraphs):
    """
    for 1 sentence paragraphs, concatenate them to the next or previous paragraph depending on context
    """
    for i in range(len(paragraphs)):
        if i >= len((paragraphs)): break
        parag = paragraphs[i]
        sentences = get_sentences(parag)
        
        if len(sentences) == 1:
            prev_paragraph = paragraphs[i-1] if i-1 >= 0 else None
            next_paragraph = paragraphs[i+1] if i+1 < len(paragraphs) else None

            if (next_paragraph and len(get_sentences(next_paragraph)) == 1):
                # If a series of 1 sentence length paragraphs exist, conc all of them in one paragraph
                while True:
                    if next_paragraph and len(get_sentences(next_paragraph)) == 1:
                        parag = conc_paragraphs(parag, next_paragraph)
                        paragraphs[i] = parag
                        del paragraphs[i+1]

                        # reinitialize for loop
                        next_paragraph = paragraphs[i+1] if i+1 < len(paragraphs) else None
                    else:
                        break

            # otherwise, just concatenate the 1 sentence paragraph to the previous paragraph
            elif prev_paragraph:
#                 print '============== Found prev eligible paragraph'
                prev_paragraph = conc_paragraphs(prev_paragraph, parag)
                paragraphs[i-1] = prev_paragraph
                del paragraphs[i]

            # if this is the first paragraph, then just concatenate it with the next one
            elif next_paragraph:
                parag = conc_paragraphs(parag, next_paragraph)
                paragraphs[i] = parag
                del paragraphs[i+1]

def get_adjusted_paragraphs(root, conc_sentences=True):
    paragraphs = get_paragraphs(root)
    if conc_sentences:
        concatenate_sentences_to_paragraphs(paragraphs)
    return paragraphs

In [55]:
def get_patent(doc_id):
    url_to_fetch = ES_URL.format(doc_id)

    response = urllib2.urlopen(url_to_fetch)
    patent_content = response.read()

    patent_object = json.loads(patent_content)['_source']
    return patent_object

# Actual Extraction

In [57]:
ABSTRACT_ID = "{}_abstract"
DESC_ID = "{}_description"
CLAIMS_ID = "{}_claims"

ABSTRACT_PART_ID = "{}_abstract_part-{}"
DESC_PART_ID = "{}_description_part-{}"
CLAIMS_PART_ID = "{}_claims_part-{}"

In [62]:
BATCH_SIZE = 10000

preprocessed_location = "/home/local/shalaby/" + "preprocessed_data/extended_pv_abs_desc_claims_all_levels_3/"
TRAINING_PREPROCESSED_FILES_PREFIX = preprocessed_location + "extended_pv_training_docs_data_preprocessed-"
VALIDATION_PREPROCESSED_FILES_PREFIX = preprocessed_location + "extended_pv_validation_docs_data_preprocessed-"
TEST_PREPROCESSED_FILES_PREFIX = preprocessed_location + "extended_pv_test_docs_data_preprocessed-"

if not os.path.exists(preprocessed_location):
    os.makedirs(preprocessed_location)

In [63]:
NUM_ABSTRACT_PARTS = 3
NUM_DESC_PARTS = 23
NUM_CLAIMS_PARTS = 4

In [64]:
def multithreaded_extended_batch_creation(start_index):

    if os.path.exists(FILE_PREFIX + str(start_index)):
        info("Batch {} already exists, skipping..".format(start_index))
        return
    
    info("Batch creation working on {}\n".format(start_index))
    token_lines = []
    start_time = time.time()
    
    len_abs_sentences = 0
    len_desc_sentences = 0
    len_desc_paragraphs = 0
    len_claims_sentences = 0
    
    len_abs_tokens = []
    len_desc_tokens = []
    len_claims_tokens = []

    for doc_index, doc_id in enumerate(DOCS_LIST[start_index:]):
        patent_doc = get_patent(doc_id)
        
        # Abstract
        abstract = patent_doc['abstract'][0]
        root = ET.fromstring(abstract.encode('utf-8'))
        abs_paragraphs = get_adjusted_paragraphs(root)
        
        # Description
        desc = patent_doc['description'][0]
        root = ET.fromstring(desc.encode('utf-8'))
        desc_paragraphs = get_adjusted_paragraphs(root)
        
        # Claims
        claims = patent_doc['claims'][0]
        root = ET.fromstring(claims.encode('utf-8'))
        claims_paragraphs = get_adjusted_paragraphs(root, conc_sentences=False)
#         claims_paragraphs = []
#         for claim in patent_doc['claims']:
#             claims_paragraphs.append(claim.strip())

#         abstract_sentences = sum([get_sentences(abs_parag) for abs_parag in abs_paragraphs], [])
#         desc_sentences = sum([get_sentences(desc_parag) for desc_parag in desc_paragraphs], [])
#         claims_sentences = sum([get_sentences(claim_parag) for claim_parag in claims_paragraphs], [])
        

        abstract_tokens = sum([sentence_wordtokenizer(parag) for parag in abs_paragraphs], [])
        desc_tokens = sum([sentence_wordtokenizer(parag) for parag in desc_paragraphs], [])
        claims_tokens = sum([sentence_wordtokenizer(parag) for parag in claims_paragraphs], [])
        
        
        
        # lists of list of tokens
        doc_tokens_list = [doc_id]  + abstract_tokens + desc_tokens + claims_tokens
        abstract_tokens_list = [ABSTRACT_ID.format(doc_id)] + abstract_tokens
        description_tokens_list = [DESC_ID.format(doc_id)] + desc_tokens
        claims_tokens_list = [CLAIMS_ID.format(doc_id)] + claims_tokens
        
        # now add the tokens lists that will be written to the file
        token_lines.append(doc_tokens_list)
        token_lines.append(abstract_tokens_list)
        token_lines.append(description_tokens_list)
        token_lines.append(claims_tokens_list)
        
        for i in range(NUM_ABSTRACT_PARTS):
            start, end = get_doc_range(i, len(abstract_tokens), NUM_ABSTRACT_PARTS)
            token_lines.append([ABSTRACT_PART_ID.format(doc_id, i+1)] + abstract_tokens[start: end])
        
        for i in range(NUM_DESC_PARTS):
            start, end = get_doc_range(i, len(desc_tokens), NUM_DESC_PARTS)
            token_lines.append([DESC_PART_ID.format(doc_id, i+1)] + desc_tokens[start: end])    
        
        for i in range(NUM_CLAIMS_PARTS):
            start, end = get_doc_range(i, len(claims_tokens), NUM_CLAIMS_PARTS)
            token_lines.append([CLAIMS_PART_ID.format(doc_id, i+1)] + claims_tokens[start: end])
            
        if doc_index % 1000 == 0: info("Doc: {:6} -> Total Lines to write: {:8}".format(start_index + doc_index, len(token_lines)))
        if doc_index >= BATCH_SIZE - 1:
            break
    duration = time.time() - start_time
    info("Finished batch {} of size {:d} in {:.0f}m {:.0f}s".format(start_index, BATCH_SIZE, * divmod(duration, 60)))
    info("For index {}, the actual number of lines written is: {}".format(start_index, len(token_lines)))
    
    write_batch(FILE_PREFIX, token_lines, start_index)
    del token_lines
    
def get_doc_range(i, number_of_tokens, number_of_parts):
    start, end = 0,0
    if number_of_tokens < number_of_parts:
        if i==0:
            return 0, None
        else:
            return number_of_tokens,None
    if i == 0:
        start = 0
    else:
        start = (number_of_tokens / number_of_parts) * i
    if i+1 == number_of_parts:
        end = None
    else:
        end = (number_of_tokens / number_of_parts) * (i+1)
    return start, end

In [65]:
def write_batch(file_prefix, batch_lines, batch_start):
    if len(batch_lines):
        print "writing batch %d" % batch_start
        with open(file_prefix + str(batch_start), 'w') as batch_file:
            for line in batch_lines:
                batch_file.write((u" ".join(line) + "\n").encode('utf-8'))

## Training

In [66]:
DOCS_LIST = training_docs_list
FILE_PREFIX = TRAINING_PREPROCESSED_FILES_PREFIX
SAMPLE_SIZE = len(training_docs_list)

In [49]:
batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE)

In [50]:
batches

[0,
 10000,
 20000,
 30000,
 40000,
 50000,
 60000,
 70000,
 80000,
 90000,
 100000,
 110000,
 120000]

In [67]:
try:
    pool = ThreadPool(12)
    # +1 since range is end-exclusive
    batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE )
    indices = pool.map(multithreaded_extended_batch_creation, batches)
    pool.close()
    pool.terminate()
finally:
    pool.close()
    pool.terminate()

2017-03-20 11:09:54,684 : INFO : Batch creation working on 30000

2017-03-20 11:09:54,684 : INFO : Batch creation working on 40000

2017-03-20 11:09:54,689 : INFO : Batch creation working on 20000

2017-03-20 11:09:54,689 : INFO : Batch creation working on 10000

2017-03-20 11:09:54,685 : INFO : Batch creation working on 50000

2017-03-20 11:09:54,690 : INFO : Batch creation working on 100000

2017-03-20 11:09:54,690 : INFO : Batch creation working on 90000

2017-03-20 11:09:54,688 : INFO : Batch creation working on 70000

2017-03-20 11:09:54,684 : INFO : Batch creation working on 0

2017-03-20 11:09:54,694 : INFO : Batch creation working on 80000

2017-03-20 11:09:54,695 : INFO : Batch creation working on 60000

2017-03-20 11:09:54,691 : INFO : Batch creation working on 110000

2017-03-20 11:09:54,891 : INFO : Doc:  60000 -> Total Lines to write:       34
2017-03-20 11:09:54,934 : INFO : Doc:  50000 -> Total Lines to write:       34
2017-03-20 11:09:54,967 : INFO : Doc:  10000 -> Tota

writing batch 0


2017-03-20 12:00:33,894 : INFO : Finished batch 20000 of size 10000 in 50m 39s
2017-03-20 12:00:33,914 : INFO : For index 20000, the actual number of lines written is: 340000


writing batch 20000


2017-03-20 12:01:04,141 : INFO : Batch creation working on 120000

2017-03-20 12:01:04,594 : INFO : Doc: 120000 -> Total Lines to write:       34
2017-03-20 12:01:15,751 : INFO : Finished batch 10000 of size 10000 in 51m 21s
2017-03-20 12:01:15,777 : INFO : For index 10000, the actual number of lines written is: 340000


...

writing batch 90000


2017-03-20 12:04:37,227 : INFO : Finished batch 100000 of size 10000 in 54m 43s
2017-03-20 12:04:37,230 : INFO : For index 100000, the actual number of lines written is: 340000


writing batch 100000


2017-03-20 12:04:55,472 : INFO : Finished batch 110000 of size 10000 in 55m 1s
2017-03-20 12:04:55,475 : INFO : For index 110000, the actual number of lines written is: 340000


writing batch 110000


In [25]:
multithreaded_extended_batch_creation(0)

2017-03-16 10:44:02,083 : INFO : Batch creation working on 0

2017-03-16 10:44:02,175 : INFO : Doc:      0 -> Total Lines to write:       78
2017-03-16 10:44:18,155 : INFO : Finished batch 0 of size 100 in 0m 16s
2017-03-16 10:44:18,160 : INFO : For index 0, the actual number of lines written is: 7800


writing batch 0


## Validation

In [68]:
DOCS_LIST = validation_docs_list
FILE_PREFIX = VALIDATION_PREPROCESSED_FILES_PREFIX
SAMPLE_SIZE = len(validation_docs_list)

In [69]:
try:
    pool = ThreadPool(11)
    # +1 since range is end-exclusive
    batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE )
    indices = pool.map(multithreaded_extended_batch_creation, batches)
    pool.close()
    pool.terminate()
finally:
    pool.close()
    pool.terminate()

2017-03-20 12:05:47,774 : INFO : Batch creation working on 0

2017-03-20 12:05:47,775 : INFO : Batch creation working on 20000

2017-03-20 12:05:47,775 : INFO : Batch creation working on 10000

2017-03-20 12:05:47,893 : INFO : Doc:      0 -> Total Lines to write:       34
2017-03-20 12:05:47,929 : INFO : Doc:  20000 -> Total Lines to write:       34
2017-03-20 12:05:48,042 : INFO : Doc:  10000 -> Total Lines to write:       34
2017-03-20 12:09:09,861 : INFO : Doc:   1000 -> Total Lines to write:    34034
2017-03-20 12:09:19,943 : INFO : Doc:  21000 -> Total Lines to write:    34034
2017-03-20 12:09:20,645 : INFO : Doc:  11000 -> Total Lines to write:    34034
2017-03-20 12:12:16,121 : INFO : Doc:   2000 -> Total Lines to write:    68034
2017-03-20 12:12:48,471 : INFO : Doc:  22000 -> Total Lines to write:    68034
2017-03-20 12:13:07,367 : INFO : Doc:  12000 -> Total Lines to write:    68034
2017-03-20 12:15:29,959 : INFO : Doc:   3000 -> Total Lines to write:   102034
2017-03-20 12:16

writing batch 0


2017-03-20 12:38:35,937 : INFO : Doc:  29000 -> Total Lines to write:   306034
2017-03-20 12:40:52,729 : INFO : Finished batch 10000 of size 10000 in 35m 5s
2017-03-20 12:40:52,734 : INFO : For index 10000, the actual number of lines written is: 340000


writing batch 10000


2017-03-20 12:40:56,764 : INFO : Finished batch 20000 of size 10000 in 35m 9s
2017-03-20 12:40:56,766 : INFO : For index 20000, the actual number of lines written is: 328950


writing batch 20000


## Test

In [18]:
DOCS_LIST = test_docs_list
FILE_PREFIX = TEST_PREPROCESSED_FILES_PREFIX
SAMPLE_SIZE = len(test_docs_list)

In [ ]:
try:
    pool = ThreadPool(16)
    # +1 since range is end-exclusive
    batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE )
    indices = pool.map(multithreaded_extended_batch_creation, batches)
    pool.close()
    pool.terminate()
finally:
    pool.close()
    pool.terminate()

2017-03-08 04:30:26,423 : INFO : Batch creation working on 0

2017-03-08 04:30:26,423 : INFO : Batch creation working on 20000

2017-03-08 04:30:26,423 : INFO : Batch creation working on 10000

2017-03-08 04:30:26,423 : INFO : Batch creation working on 30000

2017-03-08 04:30:48,394 : INFO : Doc:  10000 -> Total Lines to write:      105
2017-03-08 04:30:48,491 : INFO : Doc:      0 -> Total Lines to write:       86
2017-03-08 04:30:48,639 : INFO : Doc:  20000 -> Total Lines to write:      188
2017-03-08 04:30:53,858 : INFO : Doc:  30000 -> Total Lines to write:      514


## Stats

In [37]:
BATCH_SIZE = 1000

In [38]:
def multithreaded_extended_batch_creation(start_index):

#     if os.path.exists(FILE_PREFIX + str(start_index)):
#         info("Batch {} already exists, skipping..".format(start_index))
#         return
    
    info("Batch creation working on {}\n".format(start_index))
    token_lines = []
    start_time = time.time()
    
    len_abs_sentences = 0
    len_desc_sentences = 0
    len_desc_paragraphs = 0
    len_claims_sentences = 0
    
    len_abs_tokens = []
    len_desc_tokens = []
    len_claims_tokens = []
    
    
    len_desc_parag_tokens = []

    len_claims_paragraphs = []
    
    for doc_index, doc_id in enumerate(DOCS_LIST[start_index:]):
        patent_doc = get_patent(doc_id)
        
        # Abstract
        abstract = patent_doc['abstract'][0]
        root = ET.fromstring(abstract.encode('utf-8'))
        abs_paragraphs = get_adjusted_paragraphs(root)
        
        # Description
        desc = patent_doc['description'][0]
        root = ET.fromstring(desc.encode('utf-8'))
        desc_paragraphs = get_adjusted_paragraphs(root)
        
        # Claims
        claims = patent_doc['claims'][0]
        root = ET.fromstring(claims.encode('utf-8'))
        claims_paragraphs = get_adjusted_paragraphs(root, conc_sentences=False)
#         claims_paragraphs = []
#         for claim in patent_doc['claims']:
#             claims_paragraphs.append(claim.strip())

        len_claims_paragraphs.append(len(claims_paragraphs))
    
        abstract_sentences = sum([get_sentences(abs_parag) for abs_parag in abs_paragraphs], [])
        desc_sentences = sum([get_sentences(desc_parag) for desc_parag in desc_paragraphs], [])
        claims_sentences = sum([get_sentences(claim_parag) for claim_parag in claims_paragraphs], [])
        
        len_abs_sentences += len(abstract_sentences)
        len_desc_sentences += len(desc_sentences)
        len_claims_sentences += len(claims_sentences)
        
        len_desc_paragraphs += len(desc_paragraphs)
        

        abstract_tokens = sum([sentence_wordtokenizer(parag) for parag in abs_paragraphs], [])
        
        len_desc_parag_tokens.extend([len(sentence_wordtokenizer(parag)) for parag in desc_paragraphs])
        desc_tokens = sum([sentence_wordtokenizer(parag) for parag in desc_paragraphs], [])
        
        claims_tokens = sum([sentence_wordtokenizer(parag) for parag in claims_paragraphs], [])
        
        
        len_abs_tokens.append(len(abstract_tokens))
        len_desc_tokens.append(len(desc_tokens))
        len_claims_tokens.append(len(claims_tokens))
        
        # lists of list of tokens
        doc_tokens_list = [doc_id]  + abstract_tokens + desc_tokens + claims_tokens
        abstract_tokens_list = [ABSTRACT_ID.format(doc_id)] + abstract_tokens
        description_tokens_list = [DESC_ID.format(doc_id)] + desc_tokens
        claims_tokens_list = [CLAIMS_ID.format(doc_id)] + claims_tokens
        
        # now add the tokens lists that will be written to the file
        token_lines.append(doc_tokens_list)
        token_lines.append(abstract_tokens_list)
        token_lines.append(description_tokens_list)
        token_lines.append(claims_tokens_list)
        
        if doc_index % 1000 == 0: info("Doc: {:6} -> Total Lines to write: {:8}".format(start_index + doc_index, len(token_lines)))
        if doc_index >= BATCH_SIZE - 1:
            break
    duration = time.time() - start_time
    info("Finished batch {} of size {:d} in {:.0f}m {:.0f}s".format(start_index, BATCH_SIZE, * divmod(duration, 60)))
    info("For index {}, the actual number of lines written is: {}".format(start_index, len(token_lines)))
    
    print "Average Abstract Sentences: {}".format(len_abs_sentences/doc_index)
    print "Average Desc Sentences: {}".format(len_desc_sentences/doc_index)
    print "Average Desc Paragraphs: {}".format(len_desc_paragraphs/doc_index)
    print "Average Claims Sentences: {}".format(len_claims_sentences/doc_index)
    
    
    print "Abstract Tokens: Mean: {} - Median: {}".format(np.mean(len_abs_tokens), np.median(len_abs_tokens))
    print "Description Tokens: Mean: {} - Median: {}".format(np.mean(len_desc_tokens), np.median(len_desc_tokens))
    print "Claims Tokens: Mean: {} - Median: {}".format(np.mean(len_claims_tokens), np.median(len_claims_tokens))
    print "Description Paragraphs Tokens: Mean: {} - Median: {}".format(np.mean(len_desc_parag_tokens), 
                                                                        np.median(len_desc_parag_tokens))
    
    print "Claims Paragraphs: Mean: {} - Median: {}".format(np.mean(len_claims_paragraphs), 
                                                                        np.median(len_claims_paragraphs))
#     write_batch(FILE_PREFIX, token_lines, start_index)
    del token_lines

In [39]:
DOCS_LIST = training_docs_list
FILE_PREFIX = TRAINING_PREPROCESSED_FILES_PREFIX
SAMPLE_SIZE = len(training_docs_list)

In [40]:
try:
    pool = ThreadPool(8) # use just 6 because every batch requires a lot of memory
    # +1 since range is end-exclusive
    batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE*10 )
    indices = pool.map(multithreaded_extended_batch_creation, batches)
    pool.close()
    pool.terminate()
finally:
    pool.close()
    pool.terminate()

2017-03-16 01:00:43,555 : INFO : Batch creation working on 0

2017-03-16 01:00:43,555 : INFO : Batch creation working on 10000

2017-03-16 01:00:43,558 : INFO : Batch creation working on 40000

2017-03-16 01:00:43,555 : INFO : Batch creation working on 20000

2017-03-16 01:00:43,558 : INFO : Batch creation working on 50000

2017-03-16 01:00:43,559 : INFO : Batch creation working on 30000

2017-03-16 01:00:43,591 : INFO : Batch creation working on 60000

2017-03-16 01:00:43,601 : INFO : Batch creation working on 70000

2017-03-16 01:00:43,690 : INFO : Doc:  50000 -> Total Lines to write:        4
2017-03-16 01:00:43,742 : INFO : Doc:  60000 -> Total Lines to write:        4
2017-03-16 01:00:43,767 : INFO : Doc:      0 -> Total Lines to write:        4
2017-03-16 01:00:43,806 : INFO : Doc:  40000 -> Total Lines to write:        4
2017-03-16 01:00:43,846 : INFO : Doc:  20000 -> Total Lines to write:        4
2017-03-16 01:00:43,904 : INFO : Doc:  10000 -> Total Lines to write:        4
20

Average Abstract Sentences: 3
Average Desc Sentences: 236
Average Desc Paragraphs: 56
Average Claims Sentences: 19
Abstract Tokens: Mean: 118.576 - Median: 114.0
Description Tokens: Mean: 7694.354 - Median: 5256.5
Claims Tokens: Mean: 1101.352 - Median: 828.0
Description Paragraphs Tokens: Mean: 137.372194747 - Median: 113.0
Claims Paragraphs: Mean: 19.411 - Median: 16.0


2017-03-16 01:05:31,749 : INFO : Batch creation working on 80000

2017-03-16 01:05:33,147 : INFO : Doc:  80000 -> Total Lines to write:        4
2017-03-16 01:05:49,041 : INFO : Finished batch 40000 of size 1000 in 5m 5s
2017-03-16 01:05:49,060 : INFO : For index 40000, the actual number of lines written is: 4000


Average Abstract Sentences: 3
Average Desc Sentences: 261
Average Desc Paragraphs: 61
Average Claims Sentences: 17
Abstract Tokens: Mean: 119.666 - Median: 119.0
Description Tokens: Mean: 8411.443 - Median: 6107.0
Claims Tokens: Mean: 1111.596 - Median: 944.0
Description Paragraphs Tokens: Mean: 137.076788944 - Median: 113.0
Claims Paragraphs: Mean: 17.226 - Median: 16.0


2017-03-16 01:05:49,886 : INFO : Batch creation working on 90000

2017-03-16 01:05:50,033 : INFO : Doc:  90000 -> Total Lines to write:        4
2017-03-16 01:05:56,244 : INFO : Finished batch 50000 of size 1000 in 5m 13s
2017-03-16 01:05:56,248 : INFO : For index 50000, the actual number of lines written is: 4000


...

Average Abstract Sentences: 3
Average Desc Sentences: 276
Average Desc Paragraphs: 65
Average Claims Sentences: 16
Abstract Tokens: Mean: 116.039 - Median: 118.0
Description Tokens: Mean: 9215.904 - Median: 6593.5
Claims Tokens: Mean: 1066.299 - Median: 932.5
Description Paragraphs Tokens: Mean: 141.239908046 - Median: 116.0
Claims Paragraphs: Mean: 16.451 - Median: 16.0


2017-03-16 01:11:36,285 : INFO : Finished batch 100000 of size 1000 in 5m 39s
2017-03-16 01:11:36,288 : INFO : For index 100000, the actual number of lines written is: 4000


Average Abstract Sentences: 3
Average Desc Sentences: 287
Average Desc Paragraphs: 67
Average Claims Sentences: 16
Abstract Tokens: Mean: 113.675 - Median: 117.0
Description Tokens: Mean: 9544.564 - Median: 6973.5
Claims Tokens: Mean: 1108.746 - Median: 977.5
Description Paragraphs Tokens: Mean: 141.208486211 - Median: 116.0
Claims Paragraphs: Mean: 16.438 - Median: 17.0


2017-03-16 01:11:54,205 : INFO : Finished batch 110000 of size 1000 in 5m 55s
2017-03-16 01:11:54,208 : INFO : For index 110000, the actual number of lines written is: 4000


Average Abstract Sentences: 3
Average Desc Sentences: 311
Average Desc Paragraphs: 73
Average Claims Sentences: 16
Abstract Tokens: Mean: 114.923 - Median: 118.0
Description Tokens: Mean: 10124.423 - Median: 7225.5
Claims Tokens: Mean: 1053.8 - Median: 936.5
Description Paragraphs Tokens: Mean: 138.761056974 - Median: 115.0
Claims Paragraphs: Mean: 16.433 - Median: 16.0
